<a href="https://colab.research.google.com/github/sdhilip200/Sentiment-Classifier-using-spaCy-3.0-Transformers/blob/main/Spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
# Installing Spacy library

!pip install spacy==3.1.1
!pip install spacy-transformers

In [30]:
import spacy
print(spacy.__version__)  # 3.1.1

3.1.1


In [31]:
# Downloading the spaCy model "en_core_web_trf"
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 460.2 MB 9.0 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [32]:
# Importing libraries

import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers

# Storing docs in binary format
from spacy.tokens import DocBin


In [33]:
# Reading the dataset
df = pd.read_csv("Data1.csv", encoding='latin-1')
df.head()

,Sentiment,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [34]:
df['Text'][2]

'The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .'

In [35]:
df['Text'][3]

'With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .'

In [36]:
df['Text'][1]

'Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .'

In [37]:
df.shape

(4846, 2)

In [38]:
#Splitting the dataset into train and test
train = df.sample(frac = 0.8, random_state = 25)
test = df.drop(train.index)


In [39]:
# Checking the shape

print(train.shape, test.shape)

(3877, 2) (969, 2)


In [40]:
import spacy
nlp=spacy.load("en_core_web_trf")
nlp.pipe_names

['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [ ]:
#Converting the data into Spacy input format

In [41]:

train['tuples'] = train.apply(lambda row: (row['Text'],row['Sentiment']), axis=1)
train = train['tuples'].tolist()





In [42]:

test['tuples'] = test.apply(lambda row: (row['Text'],row['Sentiment']), axis=1)
test = test['tuples'].tolist()


In [43]:
train[0]

("Altia 's operating profit jumped to EUR 47 million from EUR 6.6 million .",
 'positive')

In [44]:
def document(data):
  text = []
  for doc, label in nlp.pipe(data, as_tuples = True):
    if (label=='positive'):
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 0
    elif (label=='negative'):
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
      doc.cats['neutral']  = 0
    else:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 0
      doc.cats['neutral']  = 1
    text.append(doc)
  
  return(text)

  

In [17]:
# Calculate the time for converting into binary document for train dataset

start_time = datetime.now()

#passing the train dataset into function 'document'
train_docs = document(train)

#Creating binary document using DocBin function in spaCy
doc_bin = DocBin(docs = train_docs)

#Saving the binary document as train.spacy
doc_bin.to_disk("train.spacy")
end_time = datetime.now()

#Printing the time duration for train dataset
print('Duration: {}'.format(end_time - start_time))




Duration: 0:06:59.525907


In [45]:
# Calculate the time for converting into binary document for test dataset

start_time = datetime.now()

#passing the test dataset into function 'document'
test_docs = document(test)
doc_bin = DocBin(docs = test_docs)
doc_bin.to_disk("valid.spacy")
end_time = datetime.now()

#Printing the time duration for test dataset
print('Duration: {}'.format(end_time - start_time))




Duration: 0:01:42.256281


https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news

In [46]:
#Converting base configuration into full config file

!python -m spacy init fill-config ./base_config.cfg ./config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [47]:
start_time = datetime.now()

!python -m spacy train config.cfg --verbose  --gpu-id 0 --output ./output_updated

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-20 02:21:36,684] [INFO] Set up nlp object from config
[2021-08-20 02:21:36,693] [DEBUG] Loading corpus from path: test.spacy
[2021-08-20 02:21:36,694] [DEBUG] Loading corpus from path: train.spacy
[2021-08-20 02:21:36,694] [INFO] Pipeline: ['transformer', 'textcat']
[2021-08-20 02:21:36,698] [INFO] Created vocabulary
[2021-08-20 02:21:36,699] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are ini

In [52]:
text = "Australia’s largest airline temporarily lays off 2,500 employees"

In [53]:
#Test the data from the best model
nlp = spacy.load("output_updated/model-best")
demo = nlp(text)
print(demo.cats)

{'positive': 0.0011619756696745753, 'negative': 0.9965638518333435, 'neutral': 0.0022742082364857197}


In [54]:
text1 = "Apple earnings: Huge iPhone 12 sales beat analyst expectations"
demo = nlp(text1)
print(demo.cats)

{'positive': 0.998343825340271, 'negative': 0.0014156486140564084, 'neutral': 0.00024052616208791733}


In [55]:
text2 = "Tata Group is looking to enter semiconductor manufacturing as part of its plans to ramp up its presence in the electronics space."
demo = nlp(text2)
print(demo.cats)

{'positive': 0.815325140953064, 'negative': 0.0041560824029147625, 'neutral': 0.1805187314748764}
